##### The program below deals with the correctenss of the syntax in propositional logic

In [4]:
# read formula. Make sure it is in prop logic
# check for predominancy in the formula

# global variables and stuff

propositions = []

def getPredSign(string,startParan):
    paran = startParan
    for i in range(len(string)):
        if string[i]==')':
            paran-=1
            if paran <0:
                return -1,-1
        elif string[i]=='(':
            paran+=1
        if (string[i]=='!' or string[i]=='=' or string[i]=='-' or string[i]=='^' or string[i]=='|' or string[i]=='∧' or string[i]=='⇔' or string[i] == '↔' or string[i]=='⇒' or string[i]=='→' or string[i]=='¬' or string[i]=='∨') and paran == 1:
            return string[i],i
    return 0,0

def getProp(string):
    global propositions
    s=set()
    for i in string:
        if i.isalpha()==True:
            s.add(i)
    propositions = list(s)

def clearSpaces(text):
    #removes the spaces from a string
    l = text.split(" ")
    text = "".join(l)
    return text

def checkParan(string):
    # checks that there is always a predSign and not more than one
    while getPredSign(string,0)!=(0,0):
        c,i = getPredSign(string,0)
        if getPredSign(string[i+1:],1) != (0,0):
            return False
        else:
            semn,poz= getPredSign(string,0)
            if semn =='!' or semn =='¬':
                return checkParan(string[2:-1])
            else:
                return (checkParan(string[1:poz]) and checkParan(string[poz+1:-1]))
    if len(string)>1:
        return False
    return True

def unicity(string):
    for i in range(len(string)-1):
        if string[i].isalpha() and string[i+1].isalpha():
            return False
    return True

def checkNrParan(string):
    nrdesc = 0
    nrinch = 0
    for i in string:
        if i == '(':
            nrdesc+=1
        elif i== ')':
            nrinch+=1
    if nrdesc == nrinch:
        return True
    else:
        return False

def is_correct(prop):
    m=checkParan(prop)
    n=unicity(prop)
    p=checkNrParan(prop)
    if p == False:
        print("The number of ( is different than the number of )")
    elif m == False:
        print("There is a problem with the predominant signs i.e. parantheses are probably missing")
    elif n == False:
        print("The sentence doesn't make sense")
   
    return m and n and p

prop = input("Proposition to be checked: ")

# deals with the task
prop = clearSpaces(prop)
getProp(prop)
print(is_correct(prop))

Proposition to be checked: ((¬ ( P ⇒ Q )) ⇔ (( P ∨ Q ) ∧ ( (¬ P) ⇒ Q )))
True


#### This one here deals with building the truth table, validity and satisfiability and also with the normal form

In [6]:
#          LEGEND
# [use the signs below or the ones in the homework]
#  ~ -> equivalence between expressions
#  = -> equivalence as connective
#  - -> implies
#  ! -> not
#  ^ -> and/conjunction
#  | -> or/disjunction

# * - true
# # - false

# Notes: use only one letter for propositions, the program can't handle large amounts of them anyway
#        we assume the expression is correct, so make sure it is and THAT IS AN EQUIVALENCE

# -------------------------table stuff-----------------------------------
import pandas as pd
import os

dictionary = {}
debouncer = False # in case of a syntax error, so we alert the user only once after proceeding
current_line = 0

# -------------------------reading stuff--------------------------------
string = input("Your expression: ")
propositions = []
values_of_propositions = []


def getProp(string):
    global propositions
    s=set()
    for i in string:
        if i.isalpha()==True:
            s.add(i)
    propositions = list(s)
            
getProp(string)

# ------------------------helpful functions-----------------------------
def clearSpaces(text):
    #removes the spaces from a string
    l = text.split(" ")
    text = "".join(l)
    return text

def backtrack(listaformata):
    global propositions
    # I shall treat this as a binary tree ;)
    if len(listaformata)==len(propositions):
        values_of_propositions.append(listaformata)
    else:
        k = listaformata[:]
        k.append(True)
        backtrack(k)
        m = listaformata[:]
        m.append(False)
        backtrack(m)
        
def cutHalf(l):
    n=len(l)
    i=0
    while i < n:
        l.pop(i)
        i+=1
        n=len(l)
    return l
        
#---------------------------------------------------------------------------

#                  VARIANTA 2 = GOD MODE
# check if stuff is a proposition. 
#           yes -> return truth value
#           no  -> check for negation, return truth value of function(restu)
#           no2 -> check for conjunction, return truth value of conjunction(function(rest1),function(rest2))
#           no3 -> check for disjunction, return truth value of disjunction(function(rest1),function(rest2))
#           no4 -> check for implication, return truth value for implies(function(rest1),function(rest2))
#           no5 -> check for equivalence, return truth value for equivalence(function(rest1),function(rest2))
#           (before every subcall, create the row corresponding to what we need if that doesn't already exist)
#           (after every subcall, add value to that column)

# ----------------------  actual program logic -----------------------------
def getPredSign(string):
    paran = 0
    for i in range(len(string)):
        if string[i]==')':
            paran-=1
        elif string[i]=='(':
            paran+=1
        if (string[i]=='!' or string[i]=='=' or string[i]=='-' or string[i]=='^' or string[i]=='|' or string[i]=='∧' or string[i]=='⇔' or string[i]=='⇒' or string[i]=='¬' or string[i]=='∨') and paran == 1:
            return string[i],i
    return 0,0

def addParentheses(string):
    return '(' + string + ')'

def preCheck(string): # fixing basic syntax parentheses related mistakes
    if string[0]!='(' and len(string)>1:
        return addParentheses(string)
    elif getPredSign(addParentheses(string))==(0,0):
        return string
    else:
        return addParentheses(string)

def conjunction(A,B):
    return A and B

def disjunction(A,B):
    return A or B

def implies(A,B):
    if A==True and B == False:
        return False
    else:
        return True

def negation(A):
    return not A

def equivalence(A,B):
    if A==B:
        return True
    else:
        return False

def addToTable(string,value):
    global dictionary
    if string in dictionary.keys():
        if len(dictionary[string]) < current_line:
            #print(len(dictionary[string]),current_line+1)
            dictionary[string].append(value)
        
    else:
        #print(string)
        dictionary[string]=[]
        dictionary[string].append(value)
    
def getTruth(string,truthArray):
    global debouncer
    if getPredSign(string) == (0,0):
        # we have an atom on our hands :)
        for i in range(len(propositions)):
            if propositions[i] == string:
                return truthArray[i]
        if string == '⊤':
            return True
        elif string == '⊥':
            return False
        
        if debouncer == False:
            print("Your syntax is lacking a bit, I'll try to patch it up for you. The result may be unpredictable")
            debouncer = True
        string = addParentheses(string)
        return getTruth(string,truthArray)
        #most likely a composed formula is not encapsulated in parantheses, so Ill do that
        
    else:
        # we have a composed proposition :/
        semn= getPredSign(string)[0]
        if semn =='!' or semn =='¬':
            value = negation(getTruth(string[2:-1],truthArray))
            addToTable(string,value)
            return value
        elif semn =='^' or semn =='∧':
            A = getTruth(string[1:getPredSign(string)[1]],truthArray)
            B = getTruth(string[getPredSign(string)[1]+1:-1],truthArray)
            value = conjunction(A,B)
            addToTable(string,value)
            return value
        elif semn =='|' or semn =='∨':
            A = getTruth(string[1:getPredSign(string)[1]],truthArray)
            B = getTruth(string[getPredSign(string)[1]+1:-1],truthArray)
            value = disjunction(A,B)
            addToTable(string,value)
            return value
        elif semn =='-' or semn =='⇒':
            A = getTruth(string[1:getPredSign(string)[1]],truthArray)
            B = getTruth(string[getPredSign(string)[1]+1:-1],truthArray)
            value = implies(A,B)
            addToTable(string,value)
            return value
        elif semn =='=' or semn =='⇔':
            A = getTruth(string[1:getPredSign(string)[1]],truthArray)
            B = getTruth(string[getPredSign(string)[1]+1:-1],truthArray)
            value = equivalence(A,B)
            addToTable(string,value)
            return value
        else:
            print("Something went wrong")
# ----------------------------------------------------------------------
string = clearSpaces(string)
string1=string
aux1 = ""
ok=False
backtrack([])
for v in values_of_propositions:
    aux1 = preCheck(string1)
    current_line+=1
    for u in range(len(propositions)):
        addToTable(propositions[u],v[u])
    u=getTruth(aux1,v)
    
#for v in dictionary.values():
#    v=cutHalf(v)

valid = True
satisfiable = False
DNF = "("
for i in range(len(dictionary[aux1])):
    if dictionary[aux1][i] == True:
        satisfiable = True
        for j in propositions:
            if dictionary[j][i]==True:
                DNF = DNF + j + " ∧ "
            else:
                DNF = DNF + "¬"+j + " ∧ "
        DNF = DNF + "\b\b) ∨ ("
    else:
        valid = False
DNF = DNF[:-4]


# ------ printing stuff

print("Valid: ", valid)
print("Satisfiable: ", satisfiable)

print("Disjunctive Normal form ", DNF)
new_dataframe2 = pd.DataFrame(
    dictionary
)

new_dataframe2

Your expression: ((((( A ⇒ ( B ∨ C)) ∧ ( (¬A) ⇒ ( C ∨ D ))) ∧ (( C ⇒ ( ¬ F)) ∧ (( ¬ C) ⇒ ( D ⇒ A )))) ∧ ( ( ¬ ( B ∧ E )) ∧ ( B ⇒ E)))⇒ (¬ ( C ⇒ F )))
Valid:  True
Satisfiable:  True
Disjunctive Normal form  (C ∧ E ∧ D ∧ A ∧ F ∧ B ) ∨ (C ∧ E ∧ D ∧ A ∧ F ∧ ¬B ) ∨ (C ∧ E ∧ D ∧ A ∧ ¬F ∧ B ) ∨ (C ∧ E ∧ D ∧ A ∧ ¬F ∧ ¬B ) ∨ (C ∧ E ∧ D ∧ ¬A ∧ F ∧ B ) ∨ (C ∧ E ∧ D ∧ ¬A ∧ F ∧ ¬B ) ∨ (C ∧ E ∧ D ∧ ¬A ∧ ¬F ∧ B ) ∨ (C ∧ E ∧ D ∧ ¬A ∧ ¬F ∧ ¬B ) ∨ (C ∧ E ∧ ¬D ∧ A ∧ F ∧ B ) ∨ (C ∧ E ∧ ¬D ∧ A ∧ F ∧ ¬B ) ∨ (C ∧ E ∧ ¬D ∧ A ∧ ¬F ∧ B ) ∨ (C ∧ E ∧ ¬D ∧ A ∧ ¬F ∧ ¬B ) ∨ (C ∧ E ∧ ¬D ∧ ¬A ∧ F ∧ B ) ∨ (C ∧ E ∧ ¬D ∧ ¬A ∧ F ∧ ¬B ) ∨ (C ∧ E ∧ ¬D ∧ ¬A ∧ ¬F ∧ B ) ∨ (C ∧ E ∧ ¬D ∧ ¬A ∧ ¬F ∧ ¬B ) ∨ (C ∧ ¬E ∧ D ∧ A ∧ F ∧ B ) ∨ (C ∧ ¬E ∧ D ∧ A ∧ F ∧ ¬B ) ∨ (C ∧ ¬E ∧ D ∧ A ∧ ¬F ∧ B ) ∨ (C ∧ ¬E ∧ D ∧ A ∧ ¬F ∧ ¬B ) ∨ (C ∧ ¬E ∧ D ∧ ¬A ∧ F ∧ B ) ∨ (C ∧ ¬E ∧ D ∧ ¬A ∧ F ∧ ¬B ) ∨ (C ∧ ¬E ∧ D ∧ ¬A ∧ ¬F ∧ B ) ∨ (C ∧ ¬E ∧ D ∧ ¬A ∧ ¬F ∧ ¬B ) ∨ (C ∧ ¬E ∧ ¬D ∧ A ∧ F ∧ B ) ∨ (C ∧ ¬E ∧ ¬D ∧ A ∧ F ∧ ¬B ) ∨ (C ∧ ¬E ∧ ¬D ∧ A ∧ ¬F ∧ B ) ∨ (C ∧

,C,E,D,A,F,B,(B∨C),(A⇒(B∨C)),(¬A),(C∨D),...,((C⇒(¬F))∧((¬C)⇒(D⇒A))),(((A⇒(B∨C))∧((¬A)⇒(C∨D)))∧((C⇒(¬F))∧((¬C)⇒(D⇒A)))),(B∧E),(¬(B∧E)),(B⇒E),((¬(B∧E))∧(B⇒E)),((((A⇒(B∨C))∧((¬A)⇒(C∨D)))∧((C⇒(¬F))∧((¬C)⇒(D⇒A))))∧((¬(B∧E))∧(B⇒E))),(C⇒F),(¬(C⇒F)),(((((A⇒(B∨C))∧((¬A)⇒(C∨D)))∧((C⇒(¬F))∧((¬C)⇒(D⇒A))))∧((¬(B∧E))∧(B⇒E)))⇒(¬(C⇒F)))
0,True,True,True,True,True,True,True,True,False,True,...,False,False,True,False,True,False,False,True,False,True
1,True,True,True,True,True,False,True,True,False,True,...,False,False,False,True,True,True,False,True,False,True
2,True,True,True,True,False,True,True,True,False,True,...,True,True,True,False,True,False,False,False,True,True
3,True,True,True,True,False,False,True,True,False,True,...,True,True,False,True,True,True,True,False,True,True
4,True,True,True,False,True,True,True,True,True,True,...,False,False,True,False,True,False,False,True,False,True
5,True,True,True,False,True,False,True,True,True,True,...,False,False,False,True,True,True,False,True,False,True
6,True,True,True,False,False,True,True,True,True,True,...,True,True,True,False,True,False,False,False,True,True
7,True,True,True,False,False,False,True,True,True,True,...,True,True,False,True,True,True,True,False,True,True
8,True,True,False,True,True,True,True,True,False,True,...,False,False,True,False,True,False,False,True,False,True
9,True,True,False,True,True,False,True,True,False,True,...,False,False,False,True,True,True,False,True,False,True
